# Procesamiento de los datos del Dataset bo222k-1173.xyz

## Exploración de los datos

### Número de átomos por muestra

In [ ]:
# Nombre del archivo
nombre_archivo = "bo222-200k-1173k.xyz"

#Encontrar número de nuestras
numero_muestras=0
with open(nombre_archivo, "r") as archivo:
    lista_atomos_en_molecula=[]
    for linea in archivo:
        linea=linea.split()
        if len(linea)==1:
            lista_atomos_en_molecula.append(linea[0])
            numero_muestras=numero_muestras+1
# Crear un conjunto de números únicos en la lista
numeros_unicos = set(lista_atomos_en_molecula)

# Iterar sobre los números únicos y contar cuántas veces aparecen en la lista
for numero in numeros_unicos:
    veces_que_aparece = lista_atomos_en_molecula.count(numero)
    print(f"{veces_que_aparece} muestras tienen {numero} átomos cada una.")

if len(numeros_unicos)==1:
    numero_atomos=int(numeros_unicos.pop())
    print("Todas las muestras tienen el mismo número de átomos, este notebook funcionara correctamente.")
else:
    print("Las muestras tienen distintos números de átomos, este notebook NO FUNCIONARÁ corretamente.")

## Ejemplo de muestra

In [ ]:
# Nombre del archivo
nombre_archivo = "bo222-200k-1173k.xyz"
numero_de_muestras_a_imprimir=2

with open(nombre_archivo, "r") as archivo:
     # Inicializamos un contador de muestras
    contador_muestras = 0
    # Iteramos sobre las líneas del archivo
    for linea in archivo:
        # Si encontramos una línea que comienza con "MD_Step", incrementamos el contador
        if linea.startswith(str(numero_atomos)):
            contador_muestras += 1
            # Imprimimos la información de la muestra
            print(f"Muestra {contador_muestras}:\n{linea.strip()}")  # strip() elimina los espacios en blanco al principio y al final
            for _ in range(numero_atomos+1):
                print(str(archivo.readline().strip()))  # Imprimimos las siguientes 4 líneas (coordinadas de átomos)

            # Salto de línea para separar las muestras en la impresión
            print()

            # Si ya hemos impreso las dos primeras muestras, salimos del bucle
            if contador_muestras == numero_de_muestras_a_imprimir:
                break


Unicamente los datos correspondientes a la energía total del arreglo (meV), las posiciones(A) y fuerzas de los átomos son de nuestro intéres(meV/A).

## Procesamiento de los datos

### Importación de módulos

In [ ]:
import numpy as np
import h5py
import random
import os
import shutil

### Creación de la carpeta auxiliar temp

In [ ]:
carpetas=["temp","datos-procesados"]
for nombre_carpeta in carpetas:
    if os.path.exists(nombre_carpeta) and os.path.isdir(nombre_carpeta):
        shutil.rmtree(nombre_carpeta)
        os.mkdir(nombre_carpeta)
    else:
        os.mkdir(nombre_carpeta)

### Funciones 

In [ ]:
#Intercamcio de elemento por su posición en la tabla periodica
def posicion_en_tabla_periodica(elemento_quimico):
    tabla_periodica = {
        "H": 1, "He": 2, "Li": 3, "Be": 4, "B": 5, "C": 6, "N": 7, "O": 8, "F": 9, "Ne": 10,
        "Na": 11, "Mg": 12, "Al": 13, "Si": 14, "P": 15, "S": 16, "Cl": 17, "Ar": 18,
        "K": 19, "Ca": 20, "Sc": 21, "Ti": 22, "V": 23, "Cr": 24, "Mn": 25, "Fe": 26, "Ni": 27, "Co": 28,
        "Cu": 29, "Zn": 30, "Ga": 31, "Ge": 32, "As": 33, "Se": 34, "Br": 35, "Kr": 36,
        "Rb": 37, "Sr": 38, "Y": 39, "Zr": 40, "Nb": 41, "Mo": 42, "Tc": 43, "Ru": 44, "Rh": 45, "Pd": 46,
        "Ag": 47, "Cd": 48, "In": 49, "Sn": 50, "Sb": 51, "Te": 52, "I": 53, "Xe": 54,
        "Cs": 55, "Ba": 56, "La": 57, "Hf": 72, "Ta": 73, "W": 74, "Re": 75, "Os": 76, "Ir": 77, "Pt": 78,
        "Au": 79, "Hg": 80, "Tl": 81, "Pb": 82, "Bi": 83, "Th": 90, "Pa": 91, "U": 92, "Np": 93, "Pu": 94,
        "Am": 95, "Cm": 96, "Bk": 97, "Cf": 98, "Es": 99, "Fm": 100
    }

    elemento = elemento_quimico.capitalize()
    if elemento in tabla_periodica:
        return tabla_periodica[elemento]
    else:
        return None

### Guardar los datos seleccionados en archivos .data

In [ ]:
# Nombre del archivo
nombre_archivo = "bo222-200k-1173k.xyz"

#Encontrar número de nuestras
with open(nombre_archivo, "r") as archivo:
    for linea in archivo:
        linea=linea.split()
        if len(linea)==1:
            continue
        if len(linea)==5:
            n_energy=[linea[-1]]
            with open("temp/energy.data", "a") as file:
                line = " ".join(str(elemento) for elemento in n_energy)
                file.write(line + "\n")
        if len(linea)==7:
            z=posicion_en_tabla_periodica(linea[0])
            n_type=[z]
            n_pos=[linea[1],linea[2],linea[3]]
            n_force=[linea[4],linea[5],linea[6]]
            with open("temp/pos.data", "a") as file:
                line = " ".join(str(elemento) for elemento in n_pos)
                file.write(line + "\n")
            with open("temp/type.data", "a") as file:
                line = " ".join(str(elemento) for elemento in n_type)
                file.write(line + "\n")
            with open("temp/force.data", "a") as file:
                line = " ".join(str(elemento) for elemento in n_force)
                file.write(line + "\n")
print("Los datos seleccionados han sido guardados")

### Cargar datos en numpy arrays

In [ ]:
# Leer los datos del archivo "energy.data"
with open("temp/energy.data", "r") as f:
    lines = f.readlines()
# Procesar los datos y crear un arreglo NumPy
data = []
for line in lines:
    values = line.strip().split()
    data.append([float(val) for val in values])
# Convertir la lista en un arreglo NumPy
energy = np.array(data)

# Leer los datos del archivo "type.data"
with open("temp/type.data", "r") as f:
    lines = f.readlines()
# Procesar los datos y crear un arreglo NumPy
data = []
for line in lines:
    values = line.strip().split()
    data.append([float(val) for val in values])
# Convertir la lista en un arreglo NumPy
types = np.array(data)

# Leer los datos del archivo "pos.data"
with open("temp/pos.data", "r") as f:
    lines = f.readlines()
# Procesar los datos y crear un arreglo NumPy
data = []
for line in lines:
    values = line.strip().split()
    data.append([float(val) for val in values])
# Convertir la lista en un arreglo NumPy
pos = np.array(data)

# Leer los datos del archivo "force.data"
with open("temp/force.data", "r") as f:
    lines = f.readlines()
# Procesar los datos y crear un arreglo NumPy
data = []
for line in lines:
    values = line.strip().split()
    data.append([float(val) for val in values])
# Convertir la lista en un arreglo NumPy
forces = np.array(data)

print("\narray's creados")

### Convertir los datos a las dimensiones correspondientes

In [ ]:
# Acomodo de dimensiones
energy=np.reshape(energy,(numero_muestras))
pos=np.reshape(pos,(numero_muestras,numero_atomos,3))
types=np.reshape(types,(numero_muestras,numero_atomos))
forces=np.reshape(forces,(numero_muestras,numero_atomos,3))

print("\nDimensiones acomodadas")

### Guardar en archivo .h5 (sin fuerzas)

In [ ]:
########################################################
filename = "datos-procesados/bo222-200k-1173k.h5"##
########################################################

f=h5py.File(filename, "w")

##############################
group_name = "bo222-200k-1173k"##
##############################

group = f.create_group(group_name)

# Guardar datos en el grupo
group.create_dataset("types", data=types)
group.create_dataset("pos", data=pos)
group.create_dataset("energy", data=energy)

f.close()

### Guardar en archivo .h5 (con fuerzas)

In [ ]:
########################################################
filename = "datos-procesados/bo222-200k-1173k-f.h5"##
########################################################

f=h5py.File(filename, "w")

##############################
group_name = "bo222-200k-1173k"##
##############################

group = f.create_group(group_name)

# Guardar datos en el grupo
group.create_dataset("types", data=types)
group.create_dataset("pos", data=pos)
group.create_dataset("energy", data=energy)
group.create_dataset("forces", data=forces)

f.close()

### Eliminación de la carpeta auxiliar

In [ ]:
nombre_carpeta="temp"
if os.path.exists(nombre_carpeta) and os.path.isdir(nombre_carpeta):
    shutil.rmtree(nombre_carpeta)